In [ ]:
import os
import pandas as pd
import shutil
import re

# Paths
metadata_path = '/media/carol/Data/DATASETS/Emotion_Datasets/Emotion_Speaker_Sets/IEMOCAP/test/metadata.csv'  # Replace with the actual path to the metadata file
source_directory = '/media/carol/Data/DATASETS/Emotion_Datasets/Raw_Datasets/IEMOCAP'
destination_directory = '/media/carol/Data/DATASETS/Emotion_Datasets/Emotion_Speaker_Sets/IEMOCAP_WAV/test'
# Read the metadata file
metadata = pd.read_csv(metadata_path)

# Ensure the destination directory exists
os.makedirs(destination_directory, exist_ok=True)

# Iterate through the metadata rows
for index, row in metadata.iterrows():
    filename = row['file_name']
    
    # Extract session ID from the filename
    session_match = re.search(r'Ses(\d+)', filename)
    if not session_match:
        print(f"Error: Unable to extract session ID from filename {filename}")
        continue
    session_id = f"Session{int(session_match.group(1))}"
    
    # Extract base filename
    base_filename = os.path.splitext(os.path.basename(filename))[0]
    intermediate_folder = "_".join(base_filename.split("_")[:-1])  # Remove the frame identifier (e.g., `_F000`)

    
    # Construct source WAV file path
    source_wav_path = os.path.join(source_directory, session_id, "sentences", "wav", intermediate_folder, f"{base_filename}.wav")
    
    # Construct destination path for WAV file
    destination_subdir = os.path.join(destination_directory, os.path.dirname(filename))
    destination_wav_path = os.path.join(destination_subdir, f"{base_filename}.wav")
    
    # Create subdirectory if it doesn't exist
    os.makedirs(destination_subdir, exist_ok=True)
    
    # Copy the WAV file
    if os.path.exists(source_wav_path):
        shutil.copy(source_wav_path, destination_wav_path)
    else:
        print(f"Warning: {source_wav_path} does not exist.")

# Copy the metadata file to the new directory
shutil.copy(metadata_path, os.path.join(destination_directory, os.path.basename(metadata_path)))

print("All files processed and copied successfully!")
'''
/media/carol/Data/DATASETS/Emotion_Datasets/Raw_Datasets/IEMOCAP/Session1/sentences/wav/Ses01F_impro01/Ses01F_impro01_F000.wav
'''

In [2]:
import pandas as pd

# 1. Open the CSV file
input_csv = R"D:\Documents\MASC\MSP_POD_dataset\Audios\Audios.tar\Audios\metadata.csv"
df = pd.read_csv(input_csv)

# 2. Remove the entire first column
#    This assumes the first column is at index 0.
df.drop(df.columns[0], axis=1, inplace=True)

# 3. In the column "file_name", 
#    - extract everything after the first slash
#    - change the extension from .png to .wav
def process_filename(filename):
    # Split on the first slash -> x.split('/', 1)
    # If there's no slash, we can just work with the original string
    parts = filename.split('/', 1)
    if len(parts) == 2:
        # Everything after the first slash
        filename_after_slash = parts[1]
    else:
        # No slash found
        filename_after_slash = parts[0]
    
    # Change .png to .wav
    return filename_after_slash.replace('.png', '.wav')

df['file_name'] = df['file_name'].apply(process_filename)

# 4. Save the new CSV
output_csv = r"D:\Documents\MASC\MSP_POD_dataset\Audios\Audios.tar\Audios\metadata.csv"
df.to_csv(output_csv, index=False)

print("Processing complete. New CSV saved as:", output_csv)


Processing complete. New CSV saved as: D:\Documents\MASC\MSP_POD_dataset\Audios\Audios.tar\Audios\metadata.csv


In [4]:
import pandas as pd
import os
from tqdm import tqdm  # Import tqdm for the progress bar

# Read the CSV with pandas
input_csv = r"D:\Documents\MASC\MSP_POD_dataset\Audios\Audios.tar\Audios\labels_consensus.csv"
df = pd.read_csv(input_csv)

# Initialize tqdm for the progress bar
transcripts = []
for idx, row in tqdm(df.iterrows(), total=len(df), desc="Processing rows"):
    wav_filename = row["file_name"]
    txt_filename = os.path.splitext(wav_filename)[0] + ".txt"
    txt_path = os.path.join(r"D:\Documents\MASC\MSP_POD_dataset\Transcripts", txt_filename)
    
    try:
        with open(txt_path, "r", encoding="utf-8") as f:
            content = f.read().strip()
    except FileNotFoundError:
        content = ""
    
    transcripts.append(content)

# Add the transcripts column
df["transcript"] = transcripts
# (Optionally update the file_name in the DataFrame to .txt)
# df["file_name"] = df["file_name"].str.replace(".wav", ".txt", regex=False)

# Write to a new CSV
output_csv =  r"D:\Documents\MASC\MSP_POD_dataset\Audios\Audios.tar\Audios\metadata2.csv"
df.to_csv(output_csv, index=False)
print("Processing complete. New CSV saved as:", output_csv)


Processing rows: 100%|██████████| 116221/116221 [11:57<00:00, 162.09it/s]


Processing complete. New CSV saved as: D:\Documents\MASC\MSP_POD_dataset\Audios\Audios.tar\Audios\metadata2.csv
